In [3]:
!pip install "git+https://github.com/facebookresearch/pytorch3d.git"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/pytorch3d.git to /tmp/pip-req-build-p3dqkt_c
  Running command git clone -q https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-p3dqkt_c
     |████████████████████████████████| 50 kB 4.3 MB/s 
     |████████████████████████████████| 596 kB 16.7 MB/s 
  Created wheel for pytorch3d: filename=pytorch3d-0.6.2-cp37-cp37m-linux_x86_64.whl size=31519313 sha256=5c3542bc9bbb16bb9d449bebfabe693b2960581e30e667c3a93f23c2061a4565
  Stored in directory: /tmp/pip-ephem-wheel-cache-t55478g2/wheels/db/2f/07/b84807ee4c9ffc917b90b716977d7fea8f9e841f6173150600
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=7fb3524563e967f6987064bcceaa4f05ce12fc0d8dc64b1a386f68c8fcca654c
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
Successfully built py

In [40]:
import os
import numpy as np
import json

from PIL import Image  
from csv import writer
import csv

from tensorflow import keras
import tensorflow as tf

from typing import Tuple

# import torch
# import torch.nn.functional as F
# from pytorch3d import _C
# from torch.autograd import Function

from pytorch3d.ops import box3d_overlap
import torch

In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jun  7 13:56:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
WIDTH = 730
HEIGHT = 530
CHANNELS = 3

BATCH_SIZE = 8
TRAIN_DATASET_NO = 3136

TEST_DATASET_NO = 348

SAMPLES_NO = TRAIN_DATASET_NO // BATCH_SIZE

DATA_PATH = "datasets/SUNRGB-D3D/SUNRGBD/kv2/kinect2data"
CSV_EXISTS = True


X_TRAIN_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/train_images_k2.csv'
y_TRAIN_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/train_annotations_k2.csv'
X_TEST_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/test_images_k2.csv'
y_TEST_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/test_annotations_k2.csv'
X_PREDICT_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/predict_k2.csv'

y_TRAIN_3DBB_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/train_annotations_3dbb_k2.csv'
y_TEST_3DBB_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/test_annotations_3dbb_k2.csv'


In [ ]:
# Utils methods

def compute_3d_annotations(bounding_box_3d):
    x = bounding_box_3d['X']
    y = bounding_box_3d['Z']
    vector1 = np.array([x[1]-x[0], y[1]-y[0], 0])
    coeff1 = np.linalg.norm(vector1)

    vector2 = np.array([x[2]-x[1], y[2]-y[1], 0])
    coeff2 = np.linalg.norm(vector2)

    vector2 = vector2/np.linalg.norm(vector2)

    up = np.cross(vector1,vector2)

    vector1 = vector1 * up[2] / up[2]
    vector2 = vector2 * up[2] / up[2]

    zmax =-bounding_box_3d['Ymax']
    zmin =-bounding_box_3d['Ymin']

    centroid2D = np.array([0.5 * (x[0] + x[2]), 0.5 * (y[0] + y[2])])

    bounding_box_3d['basis'] = np.array([vector1, vector2, np.array([0, 0, 1])])
    bounding_box_3d['coeffs'] = np.absolute([coeff1, coeff2, zmax-zmin]) / 2
    bounding_box_3d['centriod'] = np.array([centroid2D[0], centroid2D[1], 0.5 * (zmin + zmax)])
    # bounding_box_3d['classname'] = classname
    # bounding_box_3d['labelname'] = labelname
    # bounding_box_3d['sequenceName'] = sequenceName

    orientation = np.array([0.5 * (x[1] + x[0]), 0.5 * (y[1] + y[0])] - centroid2D)
    orientation = np.append(orientation, np.array([0]))
    bounding_box_3d['orientation'] = orientation / np.linalg.norm(orientation)

    # TODO: 2D bounding box
    
    # print(bounding_box_3d)
    return bounding_box_3d

class SUNObject3d(object):
    def __init__(self, bounding_box_3d):
        # data = line.split(' ')
        # data[1:] = [float(x) for x in data[1:]]
        # self.classname = data[0]
        # self.xmin = data[1] 
        # self.ymin = data[2]
        # self.xmax = data[1]+data[3]
        # self.ymax = data[2]+data[4]
        # self.box2d = np.array([self.xmin,self.ymin,self.xmax,self.ymax])
        self.centroid = bounding_box_3d['centriod'] # np.array([data[5],data[6],data[7]])
        self.unused_dimension = bounding_box_3d['coeffs'] # np.array([data[8],data[9],data[10]])
        self.w = bounding_box_3d['coeffs'][0]
        self.l = bounding_box_3d['coeffs'][1]
        self.h = bounding_box_3d['coeffs'][2]
        self.orientation = bounding_box_3d['orientation'] # np.zeros((3,))
        # self.orientation[0] = data[11]
        # self.orientation[1] = data[12]
        self.heading_angle = -1 * np.arctan2(self.orientation[1], self.orientation[0])

def rotz(t):
    """Rotation about the z-axis."""
    c = np.cos(t)
    s = np.sin(t)
    return np.array([[c, -s,  0],
                     [s,  c,  0],
                     [0,  0,  1]])

def compute_box_3d(obj, calib):
    ''' Takes an object and a projection matrix (P) and projects the 3d
        bounding box into the image plane.
        Returns:
            corners_2d: (8,2) array in image coord.
            corners_3d: (8,3) array in in upright depth coord.
    '''
    center = obj.centroid

    # compute rotational matrix around yaw axis
    R = rotz(-1*obj.heading_angle)
    #b,a,c = dimension
    #print R, a,b,c
    
    # 3d bounding box dimensions
    l = obj.l # along heading arrow
    w = obj.w # perpendicular to heading arrow
    h = obj.h

    # rotate and translate 3d bounding box
    x_corners = [-l,l,l,-l,-l,l,l,-l]
    y_corners = [w,w,-w,-w,w,w,-w,-w]
    z_corners = [h,h,h,h,-h,-h,-h,-h]
    corners_3d = np.dot(R, np.vstack([x_corners, y_corners, z_corners]))
    corners_3d[0,:] += center[0]
    corners_3d[1,:] += center[1]
    corners_3d[2,:] += center[2]

    # project the 3d bounding box into the image plane
    corners_2d,_ = calib.project_upright_depth_to_image(np.transpose(corners_3d))
    #print 'corners_2d: ', corners_2d
    return corners_2d, np.transpose(corners_3d)

In [65]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.
#
# This source code is licensed under the BSD-style license found in the
# LICENSE file in the root directory of this source tree.

from typing import Tuple

import torch
import torch.nn.functional as F
from pytorch3d import _C
from torch.autograd import Function


# -------------------------------------------------- #
#                  CONSTANTS                         #
# -------------------------------------------------- #
"""
_box_planes and _box_triangles define the 4- and 3-connectivity
of the 8 box corners.
_box_planes gives the quad faces of the 3D box
_box_triangles gives the triangle faces of the 3D box
"""
_box_planes = [
    [0, 1, 2, 3],
    [3, 2, 6, 7],
    [0, 1, 5, 4],
    [0, 3, 7, 4],
    [1, 2, 6, 5],
    [4, 5, 6, 7],
]
_box_triangles = [
    [0, 1, 2],
    [0, 3, 2],
    [4, 5, 6],
    [4, 6, 7],
    [1, 5, 6],
    [1, 6, 2],
    [0, 4, 7],
    [0, 7, 3],
    [3, 2, 6],
    [3, 6, 7],
    [0, 1, 5],
    [0, 4, 5],
]


def _check_coplanar(boxes: torch.Tensor, eps: float = 1e-4) -> None:
    faces = torch.tensor(_box_planes, dtype=torch.int64, device=boxes.device)
    # pyre-fixme[16]: `boxes` has no attribute `index_select`.
    verts = boxes.index_select(index=faces.view(-1), dim=1)
    B = boxes.shape[0]
    P, V = faces.shape
    # (B, P, 4, 3) -> (B, P, 3)
    v0, v1, v2, v3 = verts.reshape(B, P, V, 3).unbind(2)

    # Compute the normal
    e0 = F.normalize(v1 - v0, dim=-1)
    e1 = F.normalize(v2 - v0, dim=-1)
    normal = F.normalize(torch.cross(e0, e1, dim=-1), dim=-1)

    # Check the fourth vertex is also on the same plane
    mat1 = (v3 - v0).view(B, 1, -1)  # (B, 1, P*3)
    mat2 = normal.view(B, -1, 1)  # (B, P*3, 1)
    if not (mat1.bmm(mat2).abs() < eps).all().item():
        msg = "Plane vertices are not coplanar"
        raise ValueError(msg)

    return


def _check_nonzero(boxes: torch.Tensor, eps: float = 1e-4) -> None:
    """
    Checks that the sides of the box have a non zero area
    """
    faces = torch.tensor(_box_triangles, dtype=torch.int64, device=boxes.device)
    # pyre-fixme[16]: `boxes` has no attribute `index_select`.
    verts = boxes.index_select(index=faces.view(-1), dim=1)
    B = boxes.shape[0]
    T, V = faces.shape
    # (B, T, 3, 3) -> (B, T, 3)
    v0, v1, v2 = verts.reshape(B, T, V, 3).unbind(2)

    normals = torch.cross(v1 - v0, v2 - v0, dim=-1)  # (B, T, 3)
    face_areas = normals.norm(dim=-1) / 2

    if (face_areas < eps).any().item():
        msg = "Planes have zero areas"
        raise ValueError(msg)

    return


class _box3d_overlap(Function):
    """
    Torch autograd Function wrapper for box3d_overlap C++/CUDA implementations.
    Backward is not supported.
    """

    @staticmethod
    def forward(ctx, boxes1, boxes2):
        """
        Arguments defintions the same as in the box3d_overlap function
        """
        vol, iou = _C.iou_box3d(boxes1, boxes2)
        return vol, iou

    @staticmethod
    def backward(ctx, grad_vol, grad_iou):
        raise ValueError("box3d_overlap backward is not supported")


def my_box3d_overlap(
    boxes1: torch.Tensor, boxes2: torch.Tensor, eps: float = 1e-4
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Computes the intersection of 3D boxes1 and boxes2.

    Inputs boxes1, boxes2 are tensors of shape (B, 8, 3)
    (where B doesn't have to be the same for boxes1 and boxes1),
    containing the 8 corners of the boxes, as follows:

        (4) +---------+. (5)
            | ` .     |  ` .
            | (0) +---+-----+ (1)
            |     |   |     |
        (7) +-----+---+. (6)|
            ` .   |     ` . |
            (3) ` +---------+ (2)


    NOTE: Throughout this implementation, we assume that boxes
    are defined by their 8 corners exactly in the order specified in the
    diagram above for the function to give correct results. In addition
    the vertices on each plane must be coplanar.
    As an alternative to the diagram, this is a unit bounding
    box which has the correct vertex ordering:

    box_corner_vertices = [
        [0, 0, 0],
        [1, 0, 0],
        [1, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [1, 0, 1],
        [1, 1, 1],
        [0, 1, 1],
    ]

    Args:
        boxes1: tensor of shape (N, 8, 3) of the coordinates of the 1st boxes
        boxes2: tensor of shape (M, 8, 3) of the coordinates of the 2nd boxes
    Returns:
        vol: (N, M) tensor of the volume of the intersecting convex shapes
        iou: (N, M) tensor of the intersection over union which is
            defined as: `iou = vol / (vol1 + vol2 - vol)`
    """
    if not all((8, 3) == box.shape[1:] for box in [boxes1, boxes2]):
        raise ValueError("Each box in the batch must be of shape (8, 3)")

    # pyre-fixme[16]: `_box3d_overlap` has no attribute `apply`.
    vol, iou = _box3d_overlap.apply(boxes1, boxes2)

    return vol, iou


In [66]:
# 3D IoU Custom Metric

def iou_3D(y_true, y_pred):

    # Convert form tf tensor to numpy
    y_true_np = y_true.numpy().reshape(-1, 8, 2).astype(np.float32)
    y_pred_np = y_pred.numpy().reshape(-1, 8, 2).astype(np.float32)

    print(y_pred_np.shape)

    # Add the fictive third dimension
    y_true_np = np.append(y_true_np, np.zeros((y_true_np.shape[0], y_true_np.shape[1], 1)), axis=2)
    y_pred_np = np.append(y_pred_np, np.zeros((y_pred_np.shape[0], y_pred_np.shape[1], 1)), axis=2)


    # Reorder points to be compatible with pytorch3D requirements

    # TODO
    y_true_np_reorder = np.zeros(y_true_np.shape)
    y_pred_np_reorder = np.zeros(y_pred_np.shape)
    y_true_np_reorder[:, 0, :] = y_true_np[:, 2, :]
    y_pred_np_reorder[:, 0, :] = y_pred_np[:, 2, :]
    y_true_np_reorder[:, 1, :] = y_true_np[:, 3, :]
    y_pred_np_reorder[:, 1, :] = y_pred_np[:, 3, :]
    y_true_np_reorder[:, 2, :] = y_true_np[:, 7, :]
    y_pred_np_reorder[:, 2, :] = y_pred_np[:, 7, :]
    y_true_np_reorder[:, 3, :] = y_true_np[:, 6, :]
    y_pred_np_reorder[:, 3, :] = y_pred_np[:, 6, :]
    y_true_np_reorder[:, 4, :] = y_true_np[:, 1, :]
    y_pred_np_reorder[:, 4, :] = y_pred_np[:, 1, :]
    y_true_np_reorder[:, 5, :] = y_true_np[:, 0, :]
    y_pred_np_reorder[:, 5, :] = y_pred_np[:, 0, :]
    y_true_np_reorder[:, 6, :] = y_true_np[:, 4, :]
    y_pred_np_reorder[:, 6, :] = y_pred_np[:, 4, :]
    y_true_np_reorder[:, 7, :] = y_true_np[:, 5, :]
    y_pred_np_reorder[:, 7, :] = y_pred_np[:, 5, :]
    ''' 
            1 -------- 0
           /|         /|
          2 -------- 3 .
          | |        | |
          . 5 -------- 4
          |/         |/
          6 -------- 7
                |
                V
        (4) +---------+. (5)
            | ` .     |  ` .
            | (0) +---+-----+ (1)
            |     |   |     |
        (7) +-----+---+. (6)|
            ` .   |     ` . |
            (3) ` +---------+ (2)
    '''

    # Add the 1 value for "back" points
    y_true_np_reorder[:, 4:, :] = 1
    y_pred_np_reorder[:, 4:, :] = 1

    # Scale the value back to original dimensions
    y_true_np_reorder[:, :, 0] *= WIDTH
    y_true_np_reorder[:, :, 1] *= HEIGHT
    y_pred_np_reorder[:, :, 0] *= WIDTH
    y_pred_np_reorder[:, :, 1] *= HEIGHT

    # Convert from numpy to tf tensor
    y_true_torch_init = torch.tensor(y_true_np_reorder)
    y_pred_torch_init = torch.tensor(y_pred_np_reorder)

    # Reshape to (None, 8, 3) dimension
    y_true_torch_reshaped = torch.reshape(y_true_torch_init, (-1, 8, 3)).type(torch.float32)
    y_pred_torch_reshaped = torch.reshape(y_pred_torch_init, (-1, 8, 3)).type(torch.float32)


    # _, iou_3d = box3d_overal(torch_y_true, torch_y_pred)
    _, iou_3d = my_box3d_overlap(y_true_torch_reshaped, y_pred_torch_reshaped, 0.0)

    return iou_3d
    # return 0


In [7]:
def csv_lazy(file_name):
    with open(file_name) as f:
        r = csv.reader(f)
        
        for row in r:
            yield row

def batch_generator(X_file, batch_size):

    csv_generator = csv_lazy(X_file)

    while True:
        batch = np.array([])

        for i in range(batch_size):
            batch = np.append(batch, np.array(next(csv_generator)).astype('float') / 255.0)
        
        yield batch.reshape(batch_size, WIDTH, HEIGHT, CHANNELS).astype('float')

def label_generator(y_file, batch_size):
    csv_generator = csv_lazy(y_file)

    while True:
        batch = np.array([])

        for i in range(batch_size):
            batch = np.append(batch, next(csv_generator))
        
        yield batch.reshape(batch_size, 16).astype('float')
        
def model_generator(X_file, y_file, batch_size, limit_iteration):
    X_generator = batch_generator(X_file, batch_size)
    y_generator = label_generator(y_file, batch_size)
    
    iteration_no = 0

    while True:
        
        iteration_no += 1

        print('Iteration: ' + str(iteration_no))

        # Reset to start of file
        if iteration_no > limit_iteration:
            X_generator = batch_generator(X_file, batch_size)
            y_generator = label_generator(y_file, batch_size)
            iteration_no = 0

        X_batch = next(X_generator)
        y_batch = next(y_generator)

        yield (X_batch, y_batch)

In [9]:
base_model = keras.applications.xception.Xception(weights='imagenet', include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
loc_output = keras.layers.Dense(16)(avg)

optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.001)

model = keras.models.Model(inputs=base_model.input, outputs=loc_output)
model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [10]:
model.fit(x = model_generator(X_TRAIN_DATA_PATH, y_TRAIN_3DBB_DATA_PATH, BATCH_SIZE, SAMPLES_NO // 10),
          steps_per_epoch = SAMPLES_NO // 10,
          epochs = 10,
          verbose = 2
          )

Iteration: 1
Epoch 1/10
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
39/39 - 217s - loss: 0.0859 - accuracy: 0.3205 - 217s/epoch - 6s/step
Epoch 2/10
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
It

In [11]:
model.evaluate(
    x = model_generator(X_TEST_DATA_PATH, y_TEST_3DBB_DATA_PATH, 4, TEST_DATASET_NO // 4),
    verbose = 2,
    steps = TEST_DATASET_NO // 4 - 1
)

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
Iteration: 72
I

[0.12040738016366959, 0.15697674453258514]

In [12]:
model.predict(
    x = batch_generator(X_PREDICT_DATA_PATH, 10),
    verbose = 2,
    steps = 1
)

1/1 - 1s - 1s/epoch - 1s/step


array([[ 6.14213705e-01,  1.71641320e-01,  7.52669096e-01,
         3.64208110e-02,  1.89656377e-01,  4.00066376e-04,
         2.33848721e-01,  1.19274437e-01,  6.00410640e-01,
         4.25663680e-01,  6.30532563e-01,  6.26227438e-01,
         1.77347541e-01,  5.48247695e-01,  3.90549242e-01,
         4.17884082e-01],
       [ 8.15187097e-01,  2.69695848e-01,  8.99246573e-01,
         6.64521381e-03,  2.10049465e-01,  8.09814781e-02,
         4.55481321e-01, -4.50815298e-02,  7.25673497e-01,
         4.90368128e-01,  7.00146735e-01,  9.43110466e-01,
         8.11057836e-02,  7.04934239e-01,  4.53114420e-01,
         5.82110345e-01],
       [ 5.58211923e-01,  1.26638860e-01,  6.53414965e-01,
         4.57222052e-02,  1.59161776e-01, -1.32893234e-01,
         3.69138956e-01,  1.26265824e-01,  7.06510246e-01,
         4.19234723e-01,  7.80827999e-01,  7.53076077e-01,
         9.02095661e-02,  5.19173622e-01,  4.05497819e-01,
         4.92175370e-01],
       [ 4.01197910e-01,  2.77750313e

In [67]:
base_model = keras.applications.xception.Xception(weights='imagenet', include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
loc_output = keras.layers.Dense(16)(avg)

optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.001)

model = keras.models.Model(inputs=base_model.input, outputs=loc_output)
model.compile(loss='mse', optimizer=optimizer, metrics=[iou_3D], run_eagerly=True)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [68]:
model.fit(x = model_generator(X_TRAIN_DATA_PATH, y_TRAIN_3DBB_DATA_PATH, BATCH_SIZE, SAMPLES_NO // 10),
          steps_per_epoch = SAMPLES_NO // 10,
          epochs = 10,
          verbose = 2
          )

Iteration: 1
Epoch 1/10
Iteration: 2
(8, 8, 2)
Iteration: 3
(8, 8, 2)
Iteration: 4
(8, 8, 2)
Iteration: 5
(8, 8, 2)
Iteration: 6
(8, 8, 2)
Iteration: 7
(8, 8, 2)
Iteration: 8
(8, 8, 2)
Iteration: 9
(8, 8, 2)
Iteration: 10
(8, 8, 2)
Iteration: 11
(8, 8, 2)
Iteration: 12
(8, 8, 2)
Iteration: 13
(8, 8, 2)
Iteration: 14
(8, 8, 2)
Iteration: 15
(8, 8, 2)
Iteration: 16
(8, 8, 2)
Iteration: 17
(8, 8, 2)
Iteration: 18
(8, 8, 2)
Iteration: 19
(8, 8, 2)
Iteration: 20
(8, 8, 2)
Iteration: 21
(8, 8, 2)
Iteration: 22
(8, 8, 2)
Iteration: 23
(8, 8, 2)
Iteration: 24
(8, 8, 2)
Iteration: 25
(8, 8, 2)
Iteration: 26
(8, 8, 2)
Iteration: 27
(8, 8, 2)
Iteration: 28
(8, 8, 2)
Iteration: 29
(8, 8, 2)
Iteration: 30
(8, 8, 2)
Iteration: 31
(8, 8, 2)
Iteration: 32
(8, 8, 2)
Iteration: 33
(8, 8, 2)
Iteration: 34
(8, 8, 2)
Iteration: 35
(8, 8, 2)
Iteration: 36
(8, 8, 2)
Iteration: 37
(8, 8, 2)
Iteration: 38
(8, 8, 2)
Iteration: 39
(8, 8, 2)
Iteration: 40
(8, 8, 2)
39/39 - 200s - loss: 0.0871 - iou_3D: 0.2715 - 2

In [70]:
model.evaluate(
    x = model_generator(X_TEST_DATA_PATH, y_TEST_3DBB_DATA_PATH, 4, TEST_DATASET_NO // 4),
    verbose = 2,
    steps = TEST_DATASET_NO // 4 - 1
)

Iteration: 1
Iteration: 2
(4, 8, 2)
Iteration: 3
(4, 8, 2)
Iteration: 4
(4, 8, 2)
Iteration: 5
(4, 8, 2)
Iteration: 6
(4, 8, 2)
Iteration: 7
(4, 8, 2)
Iteration: 8
(4, 8, 2)
Iteration: 9
(4, 8, 2)
Iteration: 10
(4, 8, 2)
Iteration: 11
(4, 8, 2)
Iteration: 12
(4, 8, 2)
Iteration: 13
(4, 8, 2)
Iteration: 14
(4, 8, 2)
Iteration: 15
(4, 8, 2)
Iteration: 16
(4, 8, 2)
Iteration: 17
(4, 8, 2)
Iteration: 18
(4, 8, 2)
Iteration: 19
(4, 8, 2)
Iteration: 20
(4, 8, 2)
Iteration: 21
(4, 8, 2)
Iteration: 22
(4, 8, 2)
Iteration: 23
(4, 8, 2)
Iteration: 24
(4, 8, 2)
Iteration: 25
(4, 8, 2)
Iteration: 26
(4, 8, 2)
Iteration: 27
(4, 8, 2)
Iteration: 28
(4, 8, 2)
Iteration: 29
(4, 8, 2)
Iteration: 30
(4, 8, 2)
Iteration: 31
(4, 8, 2)
Iteration: 32
(4, 8, 2)
Iteration: 33
(4, 8, 2)
Iteration: 34
(4, 8, 2)
Iteration: 35
(4, 8, 2)
Iteration: 36
(4, 8, 2)
Iteration: 37
(4, 8, 2)
Iteration: 38
(4, 8, 2)
Iteration: 39
(4, 8, 2)
Iteration: 40
(4, 8, 2)
Iteration: 41
(4, 8, 2)
Iteration: 42
(4, 8, 2)
Iteration: 

[0.13280124962329865, 0.20576605200767517]

In [69]:
model.predict(
    x = batch_generator(X_PREDICT_DATA_PATH, 10),
    verbose = 2,
    steps = 1
)

1/1 - 1s - 947ms/epoch - 947ms/step


array([[ 0.70840424,  0.17190135,  0.70358866,  0.18734197,  0.19611421,
        -0.04036169,  0.35286453,  0.06026762,  0.70485044,  0.6751316 ,
         0.6770958 ,  0.8671371 ,  0.14619026,  0.8742563 ,  0.35056543,
         0.39088142],
       [ 0.6715781 ,  0.22441441,  0.7012673 ,  0.12716666,  0.14451468,
         0.07168275,  0.34633827,  0.21302615,  0.6342534 ,  0.6933534 ,
         0.81197274,  0.7096864 ,  0.19278419,  0.8211643 ,  0.35642433,
         0.5872937 ],
       [ 0.7414239 ,  0.17345276,  0.9663636 ,  0.17914046,  0.21937223,
         0.11946905,  0.3852618 ,  0.12337126,  0.6808593 ,  0.56807613,
         0.8019866 ,  0.94107604,  0.04817877,  0.8652521 ,  0.34620982,
         0.4999133 ],
       [ 0.48336026,  0.10040443,  0.7773884 ,  0.07375083,  0.16266413,
        -0.05387876,  0.3280232 ,  0.05319463,  0.50572145,  0.6286428 ,
         0.65647435,  0.6684358 ,  0.08695368,  0.69429046,  0.31955028,
         0.44492725],
       [ 0.858296  ,  0.15290126,  0

In [71]:
base_model = keras.applications.xception.Xception(weights='imagenet', include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
loc_output = keras.layers.Dense(16)(avg)

optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.001)

model = keras.models.Model(inputs=base_model.input, outputs=loc_output)
model.compile(loss='mse', optimizer=optimizer, metrics=[iou_3D], run_eagerly=True)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [72]:
model.fit(x = model_generator(X_TRAIN_DATA_PATH, y_TRAIN_3DBB_DATA_PATH, BATCH_SIZE, SAMPLES_NO // 4),
          steps_per_epoch = SAMPLES_NO // 4,
          epochs = 10,
          verbose = 2
          )

Iteration: 1
Epoch 1/10
Iteration: 2
(8, 8, 2)
Iteration: 3
(8, 8, 2)
Iteration: 4
(8, 8, 2)
Iteration: 5
(8, 8, 2)
Iteration: 6
(8, 8, 2)
Iteration: 7
(8, 8, 2)
Iteration: 8
(8, 8, 2)
Iteration: 9
(8, 8, 2)
Iteration: 10
(8, 8, 2)
Iteration: 11
(8, 8, 2)
Iteration: 12
(8, 8, 2)
Iteration: 13
(8, 8, 2)
Iteration: 14
(8, 8, 2)
Iteration: 15
(8, 8, 2)
Iteration: 16
(8, 8, 2)
Iteration: 17
(8, 8, 2)
Iteration: 18
(8, 8, 2)
Iteration: 19
(8, 8, 2)
Iteration: 20
(8, 8, 2)
Iteration: 21
(8, 8, 2)
Iteration: 22
(8, 8, 2)
Iteration: 23
(8, 8, 2)
Iteration: 24
(8, 8, 2)
Iteration: 25
(8, 8, 2)
Iteration: 26
(8, 8, 2)
Iteration: 27
(8, 8, 2)
Iteration: 28
(8, 8, 2)
Iteration: 29
(8, 8, 2)
Iteration: 30
(8, 8, 2)
Iteration: 31
(8, 8, 2)
Iteration: 32
(8, 8, 2)
Iteration: 33
(8, 8, 2)
Iteration: 34
(8, 8, 2)
Iteration: 35
(8, 8, 2)
Iteration: 36
(8, 8, 2)
Iteration: 37
(8, 8, 2)
Iteration: 38
(8, 8, 2)
Iteration: 39
(8, 8, 2)
Iteration: 40
(8, 8, 2)
Iteration: 41
(8, 8, 2)
Iteration: 42
(8, 8, 2)


In [73]:
model.evaluate(
    x = model_generator(X_TEST_DATA_PATH, y_TEST_3DBB_DATA_PATH, 4, TEST_DATASET_NO // 4),
    verbose = 2,
    steps = TEST_DATASET_NO // 4 - 1
)

Iteration: 1
Iteration: 2
(4, 8, 2)
Iteration: 3
(4, 8, 2)
Iteration: 4
(4, 8, 2)
Iteration: 5
(4, 8, 2)
Iteration: 6
(4, 8, 2)
Iteration: 7
(4, 8, 2)
Iteration: 8
(4, 8, 2)
Iteration: 9
(4, 8, 2)
Iteration: 10
(4, 8, 2)
Iteration: 11
(4, 8, 2)
Iteration: 12
(4, 8, 2)
Iteration: 13
(4, 8, 2)
Iteration: 14
(4, 8, 2)
Iteration: 15
(4, 8, 2)
Iteration: 16
(4, 8, 2)
Iteration: 17
(4, 8, 2)
Iteration: 18
(4, 8, 2)
Iteration: 19
(4, 8, 2)
Iteration: 20
(4, 8, 2)
Iteration: 21
(4, 8, 2)
Iteration: 22
(4, 8, 2)
Iteration: 23
(4, 8, 2)
Iteration: 24
(4, 8, 2)
Iteration: 25
(4, 8, 2)
Iteration: 26
(4, 8, 2)
Iteration: 27
(4, 8, 2)
Iteration: 28
(4, 8, 2)
Iteration: 29
(4, 8, 2)
Iteration: 30
(4, 8, 2)
Iteration: 31
(4, 8, 2)
Iteration: 32
(4, 8, 2)
Iteration: 33
(4, 8, 2)
Iteration: 34
(4, 8, 2)
Iteration: 35
(4, 8, 2)
Iteration: 36
(4, 8, 2)
Iteration: 37
(4, 8, 2)
Iteration: 38
(4, 8, 2)
Iteration: 39
(4, 8, 2)
Iteration: 40
(4, 8, 2)
Iteration: 41
(4, 8, 2)
Iteration: 42
(4, 8, 2)
Iteration: 

[0.06783846020698547, 0.35587242245674133]

In [74]:
model.predict(
    x = batch_generator(X_PREDICT_DATA_PATH, 10),
    verbose = 2,
    steps = 1
)

1/1 - 0s - 122ms/epoch - 122ms/step


array([[ 0.52439725,  0.00694223,  0.5819223 , -0.03960949,  0.03705515,
        -0.03051475,  0.32461652,  0.08050592,  0.47976762,  0.31422353,
         0.6009997 ,  0.53063333,  0.06826805,  0.48612234,  0.3196347 ,
         0.24944666],
       [ 0.5475745 ,  0.07682328,  0.66692215, -0.01073963,  0.12875624,
         0.03166921,  0.22531176,  0.01024533,  0.42946276,  0.3460224 ,
         0.4353307 ,  0.48410198,  0.19483723,  0.53294957,  0.23414579,
         0.3236212 ],
       [ 0.64604115,  0.08250783,  0.7143039 , -0.0538683 ,  0.06492774,
        -0.01281232,  0.30949077,  0.04304   ,  0.61452127,  0.3424266 ,
         0.6181215 ,  0.5248248 ,  0.1732233 ,  0.5109091 ,  0.40315524,
         0.32109475],
       [ 0.42532346, -0.07143462,  0.46927065, -0.05957805,  0.0578261 ,
         0.01938517,  0.0811834 ,  0.10477912,  0.43034235,  0.30211568,
         0.5310133 ,  0.50140375,  0.04563208,  0.47437432,  0.20452401,
         0.26324317],
       [ 0.60306907,  0.11335813,  0

In [75]:
model.save('/content/drive/MyDrive/Colab Datasets/Xception-3DBB')

FailedPreconditionError: ignored